In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import pymysql
import pickle

In [2]:
con = pymysql.connect(host="127.0.0.1", db="riot", user="root", password="root1234", charset='utf8')
jgdata = pd.read_sql("select * from jgdata", con)
jgdata

D:\kdigital\anaconda\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,jno,blue_jg_champ,bj_kill,bj_gold,bj_damageDone,bj_damageTaken,bj_level,bj_minion,red_jg_champ,rj_kill,rj_gold,rj_damageDone,rj_damageTaken,rj_level,rj_minion,result
0,1,Garen,1,4712,5898,10995,10,86,Nocturne,2,5137,11610,11880,9,9,1
1,2,MonkeyKing,7,7688,6170,10925,10,10,Evelynn,5,5653,8326,11914,8,2,0
2,3,JarvanIV,5,7802,4695,10875,10,15,Kayn,1,4576,3132,9294,9,18,0
3,4,XinZhao,4,5844,5399,14560,10,20,Diana,3,8088,5130,8263,12,33,1
4,5,Hecarim,2,4874,3059,14950,8,7,MasterYi,3,6317,5966,10898,9,7,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29543,29545,LeeSin,2,4656,4725,11207,8,11,Vi,2,6405,9044,7735,10,17,1
29544,29546,Elise,2,5387,5696,10543,8,2,Diana,2,5390,4455,6528,9,15,0
29545,29547,Kindred,3,5381,6374,7231,9,14,Diana,6,7405,8118,7708,9,10,1
29546,29548,Kayn,5,6480,8170,13120,10,3,Gragas,4,5243,8896,6957,9,15,1


In [3]:
# 데이터 타입 확인
jgdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29548 entries, 0 to 29547
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   jno             29548 non-null  int64 
 1   blue_jg_champ   29548 non-null  object
 2   bj_kill         29548 non-null  int64 
 3   bj_gold         29548 non-null  int64 
 4   bj_damageDone   29548 non-null  int64 
 5   bj_damageTaken  29548 non-null  int64 
 6   bj_level        29548 non-null  int64 
 7   bj_minion       29548 non-null  int64 
 8   red_jg_champ    29548 non-null  object
 9   rj_kill         29548 non-null  int64 
 10  rj_gold         29548 non-null  int64 
 11  rj_damageDone   29548 non-null  int64 
 12  rj_damageTaken  29548 non-null  int64 
 13  rj_level        29548 non-null  int64 
 14  rj_minion       29548 non-null  int64 
 15  result          29548 non-null  int64 
dtypes: int64(14), object(2)
memory usage: 3.6+ MB


In [4]:
# champ code 불러오기
with open('champ.pickle','rb') as f:
    champCode = pickle.load(f)

champCode[['champion_key','en_name']]

,champion_key,en_name
0,266,Aatrox
1,103,Ahri
2,84,Akali
3,166,Akshan
4,12,Alistar
...,...,...
156,221,Zeri
157,115,Ziggs
158,26,Zilean
159,142,Zoe


In [5]:
# 챔피언명 라벨링
def changeCode(x):
    for code, name in champCode[['champion_key','en_name']].iterrows():
        if x == name[1]:
            return name[0]
red_jg_champ_code = jgdata['red_jg_champ'].apply(changeCode)
blue_jg_champ_code = jgdata['blue_jg_champ'].apply(changeCode)

chmapToCode = pd.concat([red_jg_champ_code,blue_jg_champ_code],axis=1)
chmapToCode

,red_jg_champ,blue_jg_champ
0,56,86
1,28,62
2,141,59
3,131,5
4,11,120
...,...,...
29543,254,64
29544,131,60
29545,131,203
29546,79,141


In [6]:
#라벨링된 데이터로변경
labelJg = pd.concat([chmapToCode,jgdata.drop(['blue_jg_champ','red_jg_champ'], axis=1)],axis=1)

#결측치 제거
labelJg = labelJg.dropna(axis=0)

#타입변경
labelJg = labelJg.astype({'red_jg_champ':'int64'})
labelJg.info()
labelJg

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28617 entries, 0 to 29547
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   red_jg_champ    28617 non-null  int64 
 1   blue_jg_champ   28617 non-null  object
 2   jno             28617 non-null  int64 
 3   bj_kill         28617 non-null  int64 
 4   bj_gold         28617 non-null  int64 
 5   bj_damageDone   28617 non-null  int64 
 6   bj_damageTaken  28617 non-null  int64 
 7   bj_level        28617 non-null  int64 
 8   bj_minion       28617 non-null  int64 
 9   rj_kill         28617 non-null  int64 
 10  rj_gold         28617 non-null  int64 
 11  rj_damageDone   28617 non-null  int64 
 12  rj_damageTaken  28617 non-null  int64 
 13  rj_level        28617 non-null  int64 
 14  rj_minion       28617 non-null  int64 
 15  result          28617 non-null  int64 
dtypes: int64(15), object(1)
memory usage: 3.7+ MB


,red_jg_champ,blue_jg_champ,jno,bj_kill,bj_gold,bj_damageDone,bj_damageTaken,bj_level,bj_minion,rj_kill,rj_gold,rj_damageDone,rj_damageTaken,rj_level,rj_minion,result
0,56,86,1,1,4712,5898,10995,10,86,2,5137,11610,11880,9,9,1
1,28,62,2,7,7688,6170,10925,10,10,5,5653,8326,11914,8,2,0
2,141,59,3,5,7802,4695,10875,10,15,1,4576,3132,9294,9,18,0
3,131,5,4,4,5844,5399,14560,10,20,3,8088,5130,8263,12,33,1
4,11,120,5,2,4874,3059,14950,8,7,3,6317,5966,10898,9,7,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29543,254,64,29545,2,4656,4725,11207,8,11,2,6405,9044,7735,10,17,1
29544,131,60,29546,2,5387,5696,10543,8,2,2,5390,4455,6528,9,15,0
29545,131,203,29547,3,5381,6374,7231,9,14,6,7405,8118,7708,9,10,1
29546,79,141,29548,5,6480,8170,13120,10,3,4,5243,8896,6957,9,15,1


In [7]:
jgdata = labelJg.drop('jno',axis=1)
jgdata

,red_jg_champ,blue_jg_champ,bj_kill,bj_gold,bj_damageDone,bj_damageTaken,bj_level,bj_minion,rj_kill,rj_gold,rj_damageDone,rj_damageTaken,rj_level,rj_minion,result
0,56,86,1,4712,5898,10995,10,86,2,5137,11610,11880,9,9,1
1,28,62,7,7688,6170,10925,10,10,5,5653,8326,11914,8,2,0
2,141,59,5,7802,4695,10875,10,15,1,4576,3132,9294,9,18,0
3,131,5,4,5844,5399,14560,10,20,3,8088,5130,8263,12,33,1
4,11,120,2,4874,3059,14950,8,7,3,6317,5966,10898,9,7,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29543,254,64,2,4656,4725,11207,8,11,2,6405,9044,7735,10,17,1
29544,131,60,2,5387,5696,10543,8,2,2,5390,4455,6528,9,15,0
29545,131,203,3,5381,6374,7231,9,14,6,7405,8118,7708,9,10,1
29546,79,141,5,6480,8170,13120,10,3,4,5243,8896,6957,9,15,1


## 데이터분석

In [8]:
from sklearn.model_selection import train_test_split

In [25]:
x_train, x_test, y_train, y_test=train_test_split(jgdata.drop('result',axis=1), jgdata['result'], test_size=0.3)
len(x_train), len(x_test), len(y_train), len(y_test)

(20031, 8586, 20031, 8586)

In [33]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='liblinear', random_state=1, C=0.01)
model.fit(x_train, y_train)
pred = model.predict(x_test)
(pred==y_test).mean()


0.6785464709993012

In [34]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
accuracy_score(pred,y_test),precision_score(pred,y_test),recall_score(pred,y_test)

(0.6785464709993012, 0.6658133581568536, 0.6836320191158901)